# Testing Cellmage with Gemini 2.5 Flash

This notebook tests the capabilities of Cellmage with Google's Gemini 2.5 Flash model.

**Date:** April 24, 2025

## Setup and Configuration

Let's set up our development environment and import the necessary modules.

In [2]:
# Setup environment
import os
import sys
import logging

# Skip dotenv loading for testing
os.environ["CELLMAGE_SKIP_DOTENV"] = "1"

# Set up logging
logging.basicConfig(level=logging.INFO)

# Ensure the cellmage package can be imported
# Get the absolute path of the current working directory
notebook_dir = os.getcwd()
# Get the project root directory (parent of the notebook directory)
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

print(f"Notebook directory: {notebook_dir}")
print(f"Project root directory: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added path: {project_root}")

try:
    # Import cellmage
    import cellmage

    # Check version - handle case where __version__ might not be available
    try:
        print(f"Cellmage version: {cellmage.__version__}")
    except AttributeError:
        print("Cellmage imported successfully, but version information is not available")
except ModuleNotFoundError as e:
    print(f"Error importing cellmage: {e}")
    print("\nDebug information:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Python path: {sys.path}")
    print("\nTry running this notebook from the project root directory")

2025-04-24 06:22:04,631 - cellmage - INFO - Cellmage logging initialized


Notebook directory: /Users/tpinto/madpin/cellmage/notebooks
Project root directory: /Users/tpinto/madpin/cellmage
Added path: /Users/tpinto/madpin/cellmage
Cellmage version: 0.1.0


## Setting Up the Gemini LLM Client

Now we'll configure the LLM client to use Google's Gemini 2.5 Flash model.

In [3]:
from cellmage.adapters.direct_client import DirectLLMAdapter

# Create an LLM client with Gemini-2.5-flash model
# Note: You need to set GOOGLE_API_KEY in your environment
llm_client = DirectLLMAdapter(default_model="gemini-2.5-flash")

print("LLM client initialized with model: gemini-2.5-flash")

2025-04-24 06:22:04,698 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_key' = sk-L...xxmA
2025-04-24 06:22:04,699 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_base' = https://litellm.oracle.madpin.dev
2025-04-24 06:22:04,699 - cellmage.adapters.direct_client - INFO - [Override] Setting 'model' = gemini-2.5-flash


LLM client initialized with model: gemini-2.5-flash


## Creating Specialized Personas for Gemini

Let's create some specialized personas that work well with Gemini models.

In [4]:
from cellmage.resources.memory_loader import MemoryLoader
from cellmage.storage.memory_store import MemoryStore

# Create in-memory components for testing
persona_loader = MemoryLoader()
snippet_provider = MemoryLoader()
history_store = MemoryStore()

# Create specialized personas
persona_loader.add_persona(
    name="code_expert",
    system_message="You are a programming expert who specializes in writing clean, efficient code. Provide detailed explanations and always suggest best practices.",
    config={"temperature": 0.3, "top_p": 0.95},
)

persona_loader.add_persona(
    name="creative_writer",
    system_message="You are a creative writing assistant who helps draft engaging stories and content. Be imaginative and suggest unique perspectives.",
    config={"temperature": 0.9, "top_p": 0.98},
)

# List available personas
print(f"Available personas: {persona_loader.list_personas()}")

2025-04-24 06:22:04,738 - cellmage.resources.memory_loader - INFO - Added persona 'code_expert' to memory
2025-04-24 06:22:04,738 - cellmage.resources.memory_loader - INFO - Added persona 'creative_writer' to memory


Available personas: ['code_expert', 'creative_writer']


## Creating a Chat Manager with Gemini

Now let's set up a Chat Manager with the Gemini model and our personas.

In [5]:
# Create a chat manager
chat_manager = cellmage.ChatManager(
    llm_client=llm_client,
    persona_loader=persona_loader,
    snippet_provider=snippet_provider,
    history_store=history_store,
)

# Set default persona to code_expert
chat_manager.set_default_persona("code_expert")

print("Chat manager initialized with 'code_expert' persona")

2025-04-24 06:22:04,743 - cellmage.chat_manager - INFO - Initializing ChatManager
2025-04-24 06:22:04,743 - cellmage.chat_manager - INFO - ChatManager initialized
2025-04-24 06:22:04,744 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:22:04,744 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 06:22:04,745 - cellmage.chat_manager - INFO - Default persona set to 'code_expert'


Chat manager initialized with 'code_expert' persona


## Testing Code Generation with Gemini

Let's test Gemini's code generation capabilities through our chat manager.

In [6]:
# Add a challenging code generation task
response = chat_manager.chat(
    """Write a Python function that acts as a simple text-based calculator. 
    It should take a string expression like '2 + 3 * 4' and return the calculated result.
    Handle addition, subtraction, multiplication, division, and parentheses.
    Make sure to respect the order of operations.""",
    stream=True,
)

2025-04-24 06:22:04,750 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:22:04,751 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 2 messages


## Testing Creative Content Generation

Now let's switch to our creative writer persona and test content generation.

In [7]:
# Switch to creative_writer persona
chat_manager.set_default_persona("creative_writer")
print("Switched to 'creative_writer' persona")

# Generate creative content
response = chat_manager.chat(
    """Write a short story about a programmer who discovers an AI that can 
    predict the future, but only for trivial events. Make it humorous and 
    include a surprising twist at the end.""",
    stream=True,
)

2025-04-24 06:22:40,917 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.9
2025-04-24 06:22:40,918 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.98
2025-04-24 06:22:40,919 - cellmage.chat_manager - INFO - Default persona set to 'creative_writer'
2025-04-24 06:22:40,920 - cellmage.chat_manager - INFO - Sending message to LLM with 4 messages in context
2025-04-24 06:22:40,920 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 4 messages


Switched to 'creative_writer' persona


## Adding Context with Code Snippets

Let's test how Gemini performs with code snippets as context.

In [8]:
# Switch back to code_expert persona
chat_manager.set_default_persona("code_expert")

# Create a code snippet with a bug
snippet_provider.add_snippet(
    name="buggy_sort",
    content="""```python
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j], arr[j+1]  # Bug: This line is correct but there's a logical error
    return arr

# Test the function
test_array = [64, 34, 25, 12, 22, 11, 90]
sorted_array = bubble_sort(test_array)
print("Sorted array:", sorted_array)
```""",
)

# Add the snippet to the conversation
chat_manager.add_snippet("buggy_sort")

# Ask about the bug
response = chat_manager.chat(
    """There's a logical error in this bubble sort implementation. 
    The function seems to work correctly for this test case, but it's not implemented optimally. 
    Can you identify the issue and suggest a fix?""",
    stream=True,
)
response

2025-04-24 06:23:19,265 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:23:19,265 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 06:23:19,266 - cellmage.chat_manager - INFO - Default persona set to 'code_expert'
2025-04-24 06:23:19,266 - cellmage.resources.memory_loader - INFO - Added snippet 'buggy_sort' to memory
2025-04-24 06:23:19,267 - cellmage.chat_manager - INFO - Added snippet 'buggy_sort' as system message
2025-04-24 06:23:19,267 - cellmage.chat_manager - INFO - Sending message to LLM with 7 messages in context
2025-04-24 06:23:19,267 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 7 messages


''

## Comparing Gemini's Performance

Let's create a complex query and test it with temperature variations to see how Gemini's responses differ.

In [9]:
# Set a lower temperature for precise answers
import cellmage.integrations


chat_manager.set_override("temperature", 0.1)

# Complex technical question
tech_query = """Explain the differences between synchronous and asynchronous programming 
                 paradigms in Python, including when to use each and their advantages 
                 and disadvantages. Include code examples."""

# Get response with low temperature
print("===== Response with temperature = 0.1 =====")
precise_response = chat_manager.chat(tech_query, stream=False)

cellmage.integrations.ipython_magic.display(precise_response)
# Set a higher temperature for more creative answers
chat_manager.set_override("temperature", 0.9)

# Clear history to get a fresh response
chat_manager.clear_history()
chat_manager.set_default_persona("code_expert")

# Get response with high temperature
print("\n===== Response with temperature = 0.9 =====")
creative_response = chat_manager.chat(tech_query, stream=True)
# creative_response

2025-04-24 06:23:34,327 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.1
2025-04-24 06:23:34,328 - cellmage.chat_manager - INFO - Sending message to LLM with 9 messages in context
2025-04-24 06:23:34,328 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 9 messages


===== Response with temperature = 0.1 =====


'Okay, let\'s break down the bubble sort implementation you provided and identify the logical error.\n\nYou are absolutely right; the function *as written* has a critical flaw that prevents it from sorting the array at all. The fact that it might *seem* to work for the test case is likely a misunderstanding or perhaps you were expecting the correct output from a different version of the code.\n\n**The Logical Error:**\n\nThe issue lies in this line:\n\n```python\narr[j], arr[j+1] = arr[j], arr[j+1]\n```\n\nThis line is intended to perform a swap between `arr[j]` and `arr[j+1]` if `arr[j]` is greater than `arr[j+1]`. However, the way it\'s written, it\'s performing a tuple assignment where the values on the right-hand side are assigned back to the variables on the left-hand side *in their original order*.\n\nLet\'s trace it:\n1.  Python evaluates the right-hand side `(arr[j], arr[j+1])`. It gets the current values, say `(64, 34)`.\n2.  It then assigns these values to the variables on th

2025-04-24 06:23:51,040 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.9
2025-04-24 06:23:51,041 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:23:51,042 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:23:51,042 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 06:23:51,043 - cellmage.chat_manager - INFO - Default persona set to 'code_expert'
2025-04-24 06:23:51,043 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 06:23:51,044 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 3 messages



===== Response with temperature = 0.9 =====


## Testing Gemini's Contextual Understanding

Let's test Gemini's ability to maintain context over a multi-turn conversation.

In [10]:
# Clear history for a fresh conversation
chat_manager.clear_history()
chat_manager.set_default_persona("code_expert")

# Let's have a multi-turn conversation about a specific topic
print("===== First Turn =====")
chat_manager.chat("What is a recursive function in programming?", stream=False)

print("\n===== Second Turn =====")
chat_manager.chat("Can you give me an example using Python?", stream=False)

print("\n===== Third Turn =====")
chat_manager.chat("What are some ways to optimize it to avoid stack overflow?", stream=False)

print("\n===== Fourth Turn =====")
chat_manager.chat("Implement one of those optimization techniques in the example you provided.", stream=False)

2025-04-24 06:24:17,448 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:24:17,449 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:24:17,450 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 06:24:17,450 - cellmage.chat_manager - INFO - Default persona set to 'code_expert'
2025-04-24 06:24:17,451 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 06:24:17,451 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 3 messages


===== First Turn =====


2025-04-24 06:24:32,052 - cellmage.chat_manager - INFO - Sending message to LLM with 5 messages in context
2025-04-24 06:24:32,054 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 5 messages



===== Second Turn =====


2025-04-24 06:24:46,933 - cellmage.chat_manager - INFO - Sending message to LLM with 7 messages in context
2025-04-24 06:24:46,934 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 7 messages



===== Third Turn =====


2025-04-24 06:25:08,701 - cellmage.chat_manager - INFO - Sending message to LLM with 9 messages in context
2025-04-24 06:25:08,703 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 9 messages



===== Fourth Turn =====


'Okay, let\'s implement the most common and effective optimization technique in Python for avoiding stack overflow: **Converting the Recursive Solution to an Iterative Solution**.\n\nWe\'ll use the Factorial example, as it\'s straightforward to convert.\n\n**Original Recursive Factorial (prone to stack overflow for large N):**\n\n```python\n# (From previous example)\n# def recursive_factorial(n):\n#     if not isinstance(n, int) or n < 0:\n#         raise ValueError("Factorial is defined for non-negative integers only")\n#     if n == 0:\n#         return 1\n#     else:\n#         return n * recursive_factorial(n - 1)\n```\n\n**Optimized Iterative Factorial (avoids stack overflow):**\n\n```python\nimport sys\n\ndef iterative_factorial(n):\n    """\n    Calculates the factorial of a non-negative integer using iteration (a loop).\n    This avoids the risk of stack overflow for large n compared to recursion.\n\n    Args:\n        n: A non-negative integer.\n\n    Returns:\n        The fac

## Saving Multi-Turn Conversations

Let's save our multi-turn conversation and see how it's stored.

In [11]:
# Save the conversation
save_path = chat_manager.save_conversation("gemini_recursion_conversation")
print(f"Conversation saved to: {save_path}")

# Clear the history
chat_manager.clear_history()
print(f"History cleared, current message count: {len(chat_manager.get_history())}")

# Load the conversation back
if save_path:
    chat_manager.load_conversation(save_path)
    loaded_history = chat_manager.get_history()
    print(f"Conversation loaded, message count: {len(loaded_history)}")

    # Display the loaded conversation summary
    print("\n===== Loaded Conversation Summary =====")
    for i, msg in enumerate(loaded_history):
        if msg.role != "system":
            content_preview = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
            print(f"[{msg.role}] {content_preview}")

2025-04-24 06:25:19,439 - cellmage.storage.memory_store - INFO - Saved conversation to memory with ID: gemini_recursion_conversation
2025-04-24 06:25:19,441 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:25:19,442 - cellmage.history_manager - INFO - Loaded conversation from gemini_recursion_conversation with 10 messages


Conversation saved to: gemini_recursion_conversation
History cleared, current message count: 2
Conversation loaded, message count: 10

===== Loaded Conversation Summary =====
[user] What is a recursive function in programming?
[assistant] Okay, let's break down what a recursive function i...
[user] Can you give me an example using Python?
[assistant] Okay, let's look at a couple of classic examples o...
[user] What are some ways to optimize it to avoid stack o...
[assistant] Okay, let's talk about how to handle the risk of s...
[user] Implement one of those optimization techniques in ...
[assistant] Okay, let's implement the most common and effectiv...


## Conclusion

This notebook has demonstrated the capabilities of Cellmage with the Gemini 2.5 Flash model, including:

1. Setting up a Gemini model configuration
2. Creating specialized personas
3. Testing code generation capabilities
4. Testing creative content generation
5. Using code snippets to provide context
6. Testing contextual understanding across multiple conversation turns
7. Saving and loading multi-turn conversations

The Gemini 2.5 Flash model shows strong capabilities across both technical and creative tasks within the Cellmage framework.